# Tabular Data

Tabular data are data in a table format, such as an Excel spreadsheet.
We often need to work with tabular data.
*Pandas* is a powerful Python library for working with tabular data.

In this part, we will work with real-world data from the European Court of Human Rights [OpenData project](https://echr-opendata.eu/) (ECHR-OD).
ECHR-OD has data sets with cases from the European Court of Human Rights in different formats.
Here, we will work with data from the tabular data file `echr_2_0_0_structured_cases.csv`, which is in *CSV* format.
CSV is an abbreviation for "comma separated values".

## Reading CSV Files
To read the data set, we must first import the pandas library.
Then, we use pandas `read-csv()` function.

In [ ]:
import pandas as pd

cases = pd.read_csv('cases-5.csv')

```{note}
By convention, pandas is usually imported as `pd`.
```

## Examining Data
Now, we can examine the data set.
Pandas has a method `head()` which prints the first five rows of the table.

In [ ]:
display(cases.head())

```{admonition} `display` function
We can use the Jupyter Notebook function `display()` to show a nicer format than with `print()`.
```

This is a large table with 374 columns. Therefore, pandas shows only the first and last columns.
We can look at the column names.

In [ ]:
display(list(cases.columns))